In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [2]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [3]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [4]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

        if len(values)>0:  
            BranchedMatrix.append([branched,values])
#             BranchedValue.append(values)
#             print 'BranchedMatrix',BranchedMatrix

            if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
                SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
                SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
                print 'SecondModelOutput[1]',SecondModelOutput[1]
                print 'SecondModelOutput[0]',SecondModelOutput[0]
                if SecondModelOutput[1]>0.000001:
    #                 print 'bayad cut bezane'
                    output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
                    print 'output',output[0]
                    print 'output[1]',output[1]
                    lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
                    print 'lhs',lhs[0]
                    self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
                    print 'cut added'



In [5]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))

# admipex3(c)
CutCplex(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Cuts_Cliques                        -1
CPXPARAM_MIP_Cuts_Covers                         -1
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Cuts_FlowCovers                     -1
CPXPARAM_MIP_Cuts_Implied                        -1
CPXPARAM_MIP_Cuts_GUBCovers                      -1
CPXPARAM_MIP_Cuts_Gomory                         -1
CPXPARAM_MIP_Cuts_PathCut                        -1
CPXPARAM_MIP_Cuts_MIRCut                         -1
CPXPARAM_MIP_Cuts_Disjunctive                    -1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_MIP_Cuts_ZeroHalfCut                    -1
CPXPARAM_MIP_Cuts_MCFCut                         -1
CPXPARAM_MIP_Cuts_LiftProj                       -1
CPXPARAM_MIP_Cuts_LocalImplied                   -1
CPXPARAM_MIP_Cuts_BQP                            -1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 20 rows, 20 col

      1     3      703.1931     5      630.0000      696.9849       13   10.63%              x1 U      1      0      1
self.times_called 5
get_num_nodes 2
values [-0.0]
branchedNames ['x1']
branched [1]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 722 columns.
Reduced LP has 210 rows, 1558 columns, and 24972 nonzeros.
Presolve time = 0.02 sec. (2.53 ticks)
Initializing dual steep norms . . .
SecondModelOutput[1] 0.0
SecondModelOutput[0] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

      2     4      692.9549     3      630.0000      696.9849       17   10.63%              x1 D      2      0      1
self.times_called 7
get_num_nodes 3
values [1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x3', 'x7', 'x17']
branched [0, 1, 3, 7, 17]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 450 columns.
Reduced LP has 196 rows, 1350 columns, and 18756 nonzeros.
Presolve time = 0.02 sec. (1.92 ticks)
Initializing dual steep norms . . .
SecondModelOutput[1] 0.0
SecondModelOutput[0] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

      3     5      684.2746     3      630.0000      696.9849       20   10.63%             x17 U      3      2      2
self.times_called 9
get_num_nodes 4
values [1.0, -0.0, 1.0, 1.0, -0.0]
branchedNames ['x0', 'x1', 'x3', 'x7', 'x17']
branched [0, 1, 3, 7, 17]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 450 columns.
Reduced LP has 196 rows, 1350 columns, and 18756 nonzeros.
Presolve time = 0.02 sec. (1.92 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =            82.442102
Iteration:    62   Scaled dual infeas =             0.238587
Iteration:   124   Scaled dual infeas =             0.238587
Iteration:   186   Scaled dual infeas =             0.238587
Perturbation started.
Iteration:   202   Scaled dual infeas =             0.238596
Iteration:   264   Scaled dual infeas =             0.092570
Iteration:   315   Dual objective     =             0.018455
Iteration: 

      4     5      678.2222     3      630.0000      696.9849       26   10.63%             x17 D      4      2      2
self.times_called 11
get_num_nodes 5
values [1.0, -0.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x3', 'x7', 'x17', 'x18']
branched [0, 1, 3, 7, 17, 18]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 392 columns.
Reduced LP has 190 rows, 1288 columns, and 17293 nonzeros.
Presolve time = 0.02 sec. (1.77 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =            22.610522
Iteration:    62   Scaled dual infeas =             0.105891
Iteration:   124   Scaled dual infeas =             0.105891
Iteration:   186   Scaled dual infeas =             0.105891
Perturbation started.
Iteration:   202   Scaled dual infeas =             0.105901
Iteration:   264   Scaled dual infeas =             0.105787
Iteration:   324   Dual objective     =             0.0

...

     29     6      674.8406     3      633.0000      675.7060       76    6.75%             x14 D     29      3      3
self.times_called 46
get_num_nodes 30
values [1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x3', 'x4', 'x7', 'x10', 'x13', 'x14', 'x17', 'x18']
branched [0, 1, 2, 3, 4, 7, 10, 13, 14, 17, 18]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 162 columns.
Reduced LP has 145 rows, 918 columns, and 10404 nonzeros.
Presolve time = 0.02 sec. (1.07 ticks)
Initializing dual steep norms . . .
SecondModelOutput[1] 0.0
SecondModelOutput[0] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

self.times_called 49
get_num_nodes 31
values [1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x3', 'x4', 'x7', 'x10', 'x11', 'x13', 'x14', 'x16', 'x17', 'x18']
branched [0, 1, 2, 3, 4, 7, 10, 11, 13, 14, 16, 17, 18]
     31     7      666.5705     3      643.0000      675.7060       79    5.09%             x16 U     31     30      5
     32     6    infeasible            643.0000      675.7060       79    5.09%             x15 U     32     31      6
self.times_called 50
get_num_nodes 33
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0]
branchedNames ['x1', 'x2', 'x3', 'x8', 'x12', 'x13', 'x14', 'x17']
branched [1, 2, 3, 8, 12, 13, 14, 17]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 288 columns.
Reduced LP has 175 rows, 1152 columns, and 14439 nonzeros.
Presolve time = 0.02 sec. (1.48 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Sca

     33     6      652.7808     3      643.0000      675.4743       86    5.05%             x13 U     33      8      3
     34     5    infeasible            643.0000      675.4743       86    5.05%             x15 D     34     33      4
     35     4    infeasible            643.0000      672.4859       86    4.59%             x13 U     35      5      4
self.times_called 52
get_num_nodes 36
values [1.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x3', 'x7', 'x10', 'x14', 'x17', 'x18']
branched [0, 1, 3, 7, 10, 14, 17, 18]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 288 columns.
Reduced LP has 175 rows, 1152 columns, and 14448 nonzeros.
Presolve time = 0.01 sec. (1.48 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Scaled dual infeas =           114.899998
Iteration:    62   Scaled dual infeas =             1.780098
Iteration:   124   Scaled dual infeas =            

     36     5      672.4859     3      643.0000      672.4446       87    4.58%             x14 U     36      3      3
     37     4    infeasible            643.0000      672.4446       87    4.58%             x11 D     37     36      4
self.times_called 54
get_num_nodes 38
values [1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x3', 'x5', 'x7', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x17', 'x18']
branched [0, 1, 2, 3, 5, 7, 9, 10, 11, 12, 13, 14, 17, 18]
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 72 columns.
Reduced LP has 106 rows, 648 columns, and 6648 nonzeros.
Presolve time = 0.00 sec. (0.69 ticks)
Initializing dual steep norms . . .
SecondModelOutput[1] 0.0
SecondModelOutput[0] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

     40     4      651.1331     4      643.0000      660.8196       91    2.77%              x4 U     40     38      5
Elapsed time = 10.05 sec. (2.14 ticks, tree = 0.01 MB, solutions = 5)
     41     3    infeasible            643.0000      660.8196       91    2.77%             x15 U     41     40      6
     42     2    infeasible            643.0000      654.6784       91    1.82%             x16 D     42     30      5
     43     1    infeasible            643.0000      647.4080       91    0.69%             x15 D     43     10      5
     44     0    infeasible            643.0000      643.0000       91    0.00%              x4 D     44     29      4

User cuts applied:  3

Root node processing (before b&c):
  Real time             =    0.08 sec. (0.96 ticks)
Sequential b&c:
  Real time             =   10.00 sec. (1.20 ticks)
                          ------------
Total (root+branch&cut) =   10.08 sec. (2.16 ticks)
MIP_optimal
Objective value =  643.0

Column 0: Value =          